In [64]:
# Loading all of the data files, all at 100mhz or 100 samples per second
# Thanks to Katie for her files, I needed the sitting, jogging and stairs ones,
# the others are my own

import numpy as np
import csv

#This takes in the csv files and returns an array that can be used for machine learning
#also returns the header row which is the same for all of my files
def load_files(input_file):
    output_array = []
    input_headers = []
    with open(input_file, 'rb') as f:
        reader = csv.reader(f)
        input_headers = reader.next() #saves the headers
        for row in reader:
            output_array.append(row)
            
    # Prior testing gave me the size of the files so I know I can skip the first 10 seconds (1000 rows)
    # and then take the next 30,000 records to get 30 - 10 second chunks
    # never forget that it's all zero indexed
    output_array = output_array[999:30999]
    
    # Now let's reduce down to 10 second chunks
    
    
    return output_array, input_headers
    
rawWalkValues, Labels = load_files("walking2.csv")
rawSittingValues, Labels = load_files("sitting.csv")
rawCarValues, Labels = load_files("car.csv")
rawJogValues, Labels = load_files("Jogging.csv")
rawStairsValues, Labels = load_files("Steps.csv")
    


In [47]:
# Lines to test my data load into jupyter
# print Labels
# print rawStairsValues[0]

In [66]:
#Testing the length of the arrays, I should have 30 records after chunking into 10 second sections by / 1000
print "walk -",len(rawWalkValues) /1000
print "sit -",len(rawSittingValues)/1000
print "car -",len(rawCarValues)/1000
print "jog -",len(rawJogValues)/1000
print "stairs -",len(rawStairsValues)/1000


walk - 30
sit - 30
car - 30
jog - 30
stairs - 30


In [120]:
# from sklearn.feature_selection import VarianceThreshold

# # convert to right data shape, list of lists
# X = [[] for k in range(len(rawLabels))]
# for row in rawWalkValues:
#     for col in range(len(rawLabels)):
#         X[col].append(row[col])
        
# sel = VarianceThreshold(threshold=(.8*(1-.8)))
# sel.fit_transform(X)

# So this strips out the low variance stuff but doesn't help me identify the right features


In [126]:
# from sklearn.datasets import load_iris
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
# iris = load_iris()
# # X, y = rawWalkValues, rawLabels

# X = [[] for k in range(len(rawLabels))]
# for row in rawWalkValues:
#     for col in range(len(rawLabels)):
#         X[col].append(row[col])

# X = np.array(X)
# X.shape

# X_new = SelectKBest(chi2, k=2).fit_transform(X, rawLabels)
# X_new.shape

# This doesn't work for negative values which is what I have - so this selection is out

In [67]:
# So this doesn't work either since I don't have the correct data sets yet

from sklearn.feature_selection import RFE
from sklearn.svm import SVR

estimator = SVR(kernel="linear")
selector = RFE(estimator,3,step=1) # select the three most informative features

# convert to right data shape, list of lists
X = [[] for k in range(len(rawLabels))]
for row in rawWalkValues:
    for col in range(len(rawLabels)):
        X[col].append(row[col])

y = []
for i in range(16):
    y.append(i)
    
# selector = selector.fit(X[1:], rawLabels[1:])

# selector = selector.fit(X, y)

# print(RFE.support_)
# print(RFE.ranking_)

In [19]:
# selector.support_
selector.ranking_


array([1193, 1456, 1631, ...,  857,  878,  931])

In [ ]:
#Let's do some feature selection - which isn't the same as extraction
#http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#example-ensemble-plot-forest-importances-py

import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

# Build a classification task using 3 informative features
X, y = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=3,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=0,
                           shuffle=False)

# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

forest.fit(X, y)

# This doesn't work since my X and y are not right at this point

# convert to right data shape, list of lists
# X = [[] for k in range(len(rawLabels))]
# for row in rawWalkValues:
#     for col in range(len(rawLabels)):
#         X[col].append(row[col])
                     
# # X = np.array(X)
# forest.fit(X, rawLabels)

importances = forest.feature_importances_ #array with importances of each feature

idx = np.arange(0, X.shape[1]) #create an index array, with the number of features

features_to_keep = idx[importances > np.mean(importances)] #only keep features whose importance is greater than the mean importance
#should be about an array of size 3 (about)
print features_to_keep.shape

x_feature_selected = X[:,features_to_keep] #pull X values corresponding to the most important features

# print x_feature_selected
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

(3555L,)
1. feature 26924 (0.000800)
2. feature 21097 (0.000533)
3. feature 26189 (0.000533)
4. feature 305 (0.000533)
5. feature 18463 (0.000533)
6. feature 21138 (0.000533)
7. feature 26200 (0.000533)
8. feature 2122 (0.000533)
9. feature 32768 (0.000533)
10. feature 10718 (0.000533)
11. feature 29588 (0.000533)
12. feature 29589 (0.000533)
13. feature 3179 (0.000533)
14. feature 3174 (0.000533)
15. feature 26232 (0.000533)
16. feature 30738 (0.000533)
17. feature 6153 (0.000533)
18. feature 13956 (0.000533)
19. feature 27648 (0.000533)
20. feature 26172 (0.000533)
21. feature 16961 (0.000533)
22. feature 11882 (0.000533)
23. feature 32091 (0.000533)
24. feature 29310 (0.000533)
25. feature 27825 (0.000533)
26. feature 3393 (0.000533)
27. feature 17443 (0.000533)
28. feature 16460 (0.000533)
29. feature 6214 (0.000533)
30. feature 16953 (0.000533)
31. feature 9365 (0.000533)
32. feature 6265 (0.000533)
33. feature 30804 (0.000533)
34. feature 6245 (0.000533)
35. feature 21169 (0.0005

In [ ]:
# turn the raw data into a more compressed format and split into training and testing arrays
# based on all my files being a little longer than needed I'm going to drop the first 10 seconds (1000 rows)
# take the next 24 (10 second chunks or 24000 rows) as training, and then the next 6 (6000 rows) as testing
compressed = []

n = 0
temp = 0

for row in rawWalkValues:
    temp += float(row[7]) # TODO:  What column should I use? and would it be different for different files
    n +=1
    if n == 1000:
        compressed.append(temp/n)
        temp = 0
        n = 0
    
#splitting it up into two sets
train = []
test = []

i = 0
for i in range(len(compressed)):
    if i < 24:
        train.append(compressed[1])
    elif i < 30:
        test.append(compressed[1])
    



In [50]:
print train
print test

[-0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987]
[-0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987]


In [ ]:
# Where the Machine Learning happens
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

clf = tree.DecisionTreeClassifier()
  

    
# >>> from sklearn import tree
# >>> X = [[0, 0], [1, 1]] training set
# >>> Y = [0, 1] training labels
# >>> clf = tree.DecisionTreeClassifier()
# >>> clf = clf.fit(X, Y)
